In [ ]:
# sample packets given in the assignment question.

packet_list = ["37920100000100000000000106636c69656e740764726f70626f7803636f6d00000100010000290200000000000000",
"7baf818000010001000000010f7a2d7034322d696e7374616772616d046331307209696e7374616772616d03636f6d0000010001c00c000100010000000c00049df017ae0000290200000000000000",
"a0208180000100010000000105666f6e74730a676f6f676c656170697303636f6d0000010001c00c00010001000000ae00048efab64a0000290200000000000000",
"4c76010000010000000000010c74696d65736f66696e6469610a696e64696174696d657303636f6d00000100010000290200000000000000",
"629f8180000100010000000103637365046969746d02616302696e0000010001c00c0001000100001f9600040a0608020000290200000000000000",
]

In [ ]:
# given a hexadecimal octet, converts it to ASCII letter
def hex_to_letters(hex_string):
    letters = ''
    for i in range(0, len(hex_string), 2):
        hex_byte = hex_string[i:i+2]
        letter = chr(int(hex_byte, 16))
        letters += letter
    return letters

In [ ]:
# defining custom Exception classes to raise
class FormatError(Exception):
    pass

class ServerFailure(Exception):
    pass

class NameError(Exception):
    pass

class NotImplemented(Exception):
    pass

class Refused(Exception):
    pass

In [ ]:
# checking if a label of the domain name is valid according to RFC 1035
# 1.length of label <= 63
# 2. domain name starts with a letter, can end with letter or digit
# 3. domain name can have as interior characters only letters, digits, and hyphen.

def is_valid_label(label):
  if len(label) > 63:
      return False

  if not label[0].isalpha() or not label[-1].isalnum():
      return False

  for char in label[1:-1]:
      if not char.isalnum() and char!='-':
          return False

  return True

In [ ]:
def extract_dns_info(dns_packet, query_response_bit):

    # storing the header section data
    id = int(dns_packet[:4], 16)
    flags = int(dns_packet[4:8], 16)

    #  Response code
    rcode = int(dns_packet[7], 16)

    # response code based actions
    if rcode == 0:
        pass
    elif rcode == 1:
        raise FormatError("The name server was unable to interpret the query.")
    elif rcode == 2:
        raise ServerFailure("The name server was unable to process this query due to a problem with the name server.")
    elif rcode == 3:
        raise NameError("The domain name referenced in the query does not exist.")
    elif rcode == 4:
        raise NotImplemented("The name server does not support the requested kind of query.")
    elif rcode == 5:
        raise Refused("The name server refuses to perform the specified operation for policy reasons.")
    elif 6 <= rcode <= 15:
        raise ValueError("Reserved for future use - Response code:", rcode)


    question_count = int(dns_packet[8:12], 16)
    answer_count = int(dns_packet[12:16], 16)

    cursor = 24
    qname = []
    while True:
        subsequent_string_length = int(dns_packet[cursor:cursor+2], 16)

        # if you see a null octet (00), terminate reading of domain name
        if subsequent_string_length == 0:
            break

        name = dns_packet[cursor+2:cursor+2+subsequent_string_length*2]
        letters = hex_to_letters(name)

        if is_valid_label(letters):
            pass
        else:
            raise ValueError("Invalid label format:", letters)

        qname.append(letters)
        cursor += 2 + subsequent_string_length * 2
        # last two for null octet
    cursor+=2
    shifted_amount = cursor-24
    domain_name = '.'.join(qname)

    ipv4_address = 'query packet, not applicable'
    # Check if packet is of query or response type -> extract ipv4 address only if packet is of response type.

    cursor+=8
    if(query_response_bit=='1'):
      resource_record = (dns_packet[cursor:])

      # checking the leading two bits - if 11 => messsage compression used, if 00 => domain name again present/
      integer_value = int(resource_record[0], 16)
      binary_string = bin(integer_value)[2:].zfill(4)

      # message compression used
      if(binary_string[:2]=='11'):
        cursor += 24
        ipv4_string = dns_packet[cursor:cursor+8]
        ip_index = 0
        ip_list = []
        for i in range(0,4):
          part_ip = ipv4_string[ip_index:ip_index+2]
          letters = int(part_ip, 16)
          letters = str(letters)

          ip_list.append(letters)
          ip_index += 2
      elif(binary_string[:2]=='00'):
        # domain name present - no message compression
        cursor+=shifted_amount
        cursor += 20
        ipv4_string = dns_packet[cursor:cursor+8]
        ip_index = 0
        ip_list = []
        for i in range(0,4):
          part_ip = ipv4_string[ip_index:ip_index+2]
          letters = int(part_ip, 16)
          letters = str(letters)

          ip_list.append(letters)
          ip_index += 2

      else:
        raise ValueError("Not a valid DNS packet. \"label must begin with two zero bits because labels are restricted to 63 octets or less\" ")

      # message compression used => 2 octets for name
      # TYPE, CLASS, TTL, RDLENGTH => 4 octets each
      # RDLENGTH -> 4 octet - for ipv4 address

      ipv4_address = '.'.join(ip_list)

    return domain_name, ipv4_address

In [ ]:
def dns_extract(dns_packet):
  integer_value = int(dns_packet[4],16)
  binary_string = bin(integer_value)[2:].zfill(4)
  query_response_bit = binary_string[0]

  domain_name, ipv4_address = extract_dns_info(dns_packet, query_response_bit)

  print("Domain Name:", domain_name)
  print("IPv4 Address:", ipv4_address)

In [ ]:
# DNS packet
dns_packet = "629f8180000100010000000103637365046969746d02616302696e0000010001c00c0001000100001f9600040a0608020000290200000000000000"

In [ ]:
# calling the dns_extract() function. Pass the packet string as argument

dns_extract(dns_packet)

Domain Name: cse.iitm.ac.in
IPv4 Address: 10.6.8.2


In [ ]:
for item in packet_list:
  dns_extract(item)
  print()

Domain Name: client.dropbox.com
IPv4 Address: query packet, not applicable

Domain Name: z-p42-instagram.c10r.instagram.com
IPv4 Address: 157.240.23.174

Domain Name: fonts.googleapis.com
IPv4 Address: 142.250.182.74

Domain Name: timesofindia.indiatimes.com
IPv4 Address: query packet, not applicable

Domain Name: cse.iitm.ac.in
IPv4 Address: 10.6.8.2

